In [ ]:
import pandas as pd
import csv
from Bio.Seq import Seq
import random
from Bio.SeqRecord import SeqRecord

tsv to csv

In [ ]:
input_file = "../data/ecoli_only.tsv"
output_file = "sorfdb_ecoli.csv"

print("Rozpoczynam konwersję... To może potrwać kilka minut.")

with open(input_file, mode="r", encoding="utf-8") as tsv_file:
    # Używamy DictReader, aby mieć dostęp do kolumn po nazwach
    reader = csv.DictReader(tsv_file, delimiter="\t")

    # Pobieramy nagłówki (kolumny)
    fieldnames = reader.fieldnames

    with open(output_file, mode="w", encoding="utf-8", newline="") as csv_file:
        # Zapisujemy do formatu CSV (domyślnie przecinki)
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

        counter = 0
        for row in reader:
            # --- OPCJONALNIE: Biopython ---
            # Jeśli chcesz sprawdzić poprawność sekwencji nukleotydowej:
            # sequence = Seq(row['sorf'])
            # row['sorf'] = str(sequence.upper()) # Przykład operacji

            writer.writerow(row)

            # Licznik postępu
            counter += 1
            if counter % 100000 == 0:
                print(f"Przetworzono {counter} wierszy...")

print(f"Gotowe! Plik zapisany jako: {output_file}")

In [ ]:
def random_sample_tsv(input_path, output_path, k=1000):
    """
    Wybiera losowo k wierszy z pliku TSV i zapisuje do CSV.
    Metoda: Reservoir Sampling (pamięciooszczędna).
    """
    reservoir = []

    print(f"Rozpoczynam losowanie {k} wpisów z {input_path}...")

    with open(input_path, "r", encoding="utf-8") as tsv_file:
        # DictReader automatycznie pobiera nagłówki z pierwszego wiersza
        reader = csv.DictReader(tsv_file, delimiter="\t")
        headers = reader.fieldnames

        for i, row in enumerate(reader):
            # Wypełniamy reservoir pierwszymi k elementami
            if len(reservoir) < k:
                reservoir.append(row)
            else:
                # Dla każdego kolejnego elementu decydujemy, czy zastąpić nim
                # jeden z elementów w reservoir
                j = random.randint(0, i)
                if j < k:
                    reservoir[j] = row

            if (i + 1) % 500000 == 0:
                print(f"Przeskanowano {i + 1} wierszy...")

    # Używamy Biopythona do "opakowania" danych (opcjonalnie, dla walidacji)
    # i przygotowania do zapisu
    print("Konwertowanie wybranych danych...")
    processed_records = []
    for entry in reservoir:
        processed_records.append(entry)

    # Zapis do pliku CSV
    print(f"Zapisywanie do {output_path}...")
    with open(output_path, "w", encoding="utf-8", newline="") as csv_out:
        writer = csv.DictWriter(csv_out, fieldnames=headers)
        writer.writeheader()
        writer.writerows(processed_records)

    print("Zakończono pomyślnie!")


# --- URUCHOMIENIE ---
input_tsv = "../data/ecoli_only.tsv"  # Twoja ścieżka do pliku
output_csv = "sorfdb_random_15000.csv"

random_sample_tsv(input_tsv, output_csv, k=15000)